# Moving QGroundcontrol Files to Database
Taking the files from Memorizer v24 and creating a new table within kmaps database to store these files until futher notice.

## Importing

In [17]:
import pandas as pd
import mariadb
import sys
import numpy as np
import os

In [18]:
# Importing QGroundcontrol Files from Hard Drive -- Memv24
column_names = ["alloc_ip", "pid", "ptr", "size", "alloc_jiffies", "free_jiffies", "free_ip", "type", "command", "slab", "new_type"]
df = pd.read_csv('/Users/gimli/Desktop/mergeddatabase/strider.v25r1.kmap', encoding='utf-16', names = column_names)

In [19]:
# Displaying dataframe
print('Number of Rows:', df.shape[0])
df.head()

Number of Rows: 72025356


,alloc_ip,pid,ptr,size,alloc_jiffies,free_jiffies,free_ip,type,command,slab,new_type
0,ffffffff81808527,7599,ffff8881014a5500,4352,198,199,ffffffff81806806,KMEM_CACHE_BULK,bash,names_cache,NaN
1,ffffffff81808527,7599,ffff8881014a5500,4352,196,197,ffffffff81806806,KMEM_CACHE_BULK,bash,names_cache,NaN
2,ffffffff81808527,7599,ffff8881014a5500,4352,194,195,ffffffff81806806,KMEM_CACHE_BULK,bash,names_cache,NaN
3,ffffffff81808527,7599,ffff8881014a5500,4352,192,193,ffffffff81806806,KMEM_CACHE_BULK,bash,names_cache,NaN
4,ffffffff81b5a541,7599,ffff888111e8cc00,48,33,191,ffffffff81b59196,KMEM_CACHE_BULK,bash,lsm_file_cache,NaN


In [20]:
df.isna().sum()

alloc_ip                0
pid                     0
ptr                     0
size                    0
alloc_jiffies           0
free_jiffies            0
free_ip                 0
type                    0
command                 0
slab                    0
new_type         49389180
dtype: int64

## Pushing to MariaDB MySQL Server

In [21]:
def process_dataframe(df):
    try:
        conn = mariadb.connect(
            host="localhost",
            port=3306,
            user="root",
            database='kmaps',
            password="toenail"
        )
    except mariadb.Error as e:
        print(f"Error connecting to the database: {e}")
        sys.exit(1)

    cursor = conn.cursor()

    if 'new_type' in df.columns:
        df = df.drop(columns=["new_type"])

    insert_query = """
        INSERT IGNORE INTO kmapsqgroundcontrol (alloc_ip, pid, ptr, size, alloc_jiffies, free_jiffies, free_ip, type, command, slab)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    batch_size = 1000
    for start in range(0, len(df), batch_size):
        batch = df.iloc[start:start+batch_size].values.tolist()
        cursor.executemany(insert_query, batch)
        conn.commit()
        print(f"Inserted rows {start} to {start + len(batch) - 1}")

    cursor.close()
    conn.close()

if __name__ == "__main__":
    process_dataframe(df)

Inserted rows 0 to 999
Inserted rows 1000 to 1999
Inserted rows 2000 to 2999
Inserted rows 3000 to 3999
Inserted rows 4000 to 4999
Inserted rows 5000 to 5999
Inserted rows 6000 to 6999
Inserted rows 7000 to 7999
Inserted rows 8000 to 8999
Inserted rows 9000 to 9999
Inserted rows 10000 to 10999
Inserted rows 11000 to 11999
Inserted rows 12000 to 12999
Inserted rows 13000 to 13999
Inserted rows 14000 to 14999
Inserted rows 15000 to 15999
Inserted rows 16000 to 16999
Inserted rows 17000 to 17999
Inserted rows 18000 to 18999
Inserted rows 19000 to 19999
Inserted rows 20000 to 20999
Inserted rows 21000 to 21999
Inserted rows 22000 to 22999
Inserted rows 23000 to 23999
Inserted rows 24000 to 24999
Inserted rows 25000 to 25999
Inserted rows 26000 to 26999
Inserted rows 27000 to 27999
Inserted rows 28000 to 28999
Inserted rows 29000 to 29999
Inserted rows 30000 to 30999
Inserted rows 31000 to 31999
Inserted rows 32000 to 32999
Inserted rows 33000 to 33999
Inserted rows 34000 to 34999
Inserted 